In [1]:
import pandas as pd
import datetime
import plotly.express as px
from mingus.containers import Note

pd.set_option("display.max_rows", None)

In [2]:
# read dataset
df = pd.read_csv('./data/piano_sales.csv',
                 dtype = {"YEAR": int})

# format dataset
df['YEAR'] = pd.to_datetime(df['YEAR'], format = "%Y")
df['YEAR'] = df['YEAR'].dt.year
df = df.set_index('YEAR')

# update nulls to 0
df = df.fillna(0)

In [3]:
# cut total piano sales into 88 bins, one for each key on grand piano
cut_df = pd.cut(x = df['TOTALS'],
                bins = 88,
                labels = False)

In [4]:
def convert_raw_note(item):
    # Convert integer to pitch
    c = Note()
    return str(c.from_int(item))

In [5]:
# do some arithmetics to shift raw note value to appropriate values
df["note_raw"] = cut_df.apply(lambda row: row + 8)

# convert int to pitch
df['note'] = df['note_raw'].apply(convert_raw_note)

# convert null sales years to ""
df.loc[df['note_raw'] == 8,'note'] = ''

In [6]:
df.head(10)

,VERTICAL PIANOS,GRAND PIANOS,ELECTRONIC,PNEUMATIC PLAYERS,TOTALS,note_raw,note
YEAR,,,,,,,
2007,31681.0,22189.0,8666.0,0.0,62536.0,23,'B-1'
2006,39862.0,26935.0,10169.0,0.0,76966.0,26,'D-2'
2005,48527.0,35211.0,11780.0,0.0,95518.0,31,'G-2'
2004,49453.0,32377.0,11670.0,0.0,93500.0,30,'F#-2'
2003,56401.0,31433.0,9530.0,0.0,97364.0,31,'G-2'
2002,53266.0,30776.0,9550.0,0.0,93592.0,30,'F#-2'
2001,50923.0,28064.0,9470.0,0.0,88457.0,29,'F-2'
2000,60623.0,33370.0,11410.0,0.0,105403.0,33,'A-2'
1999,59946.0,32155.0,9216.0,0.0,101347.0,32,'G#-2'


In [7]:
for i in sorted(df.index):
    fig = px.bar(df.loc[i:, :],
                 y="TOTALS",
                 color = 'TOTALS',
                 color_continuous_scale = "Rainbow",
                 #hover_data=['TOTALS', 'note'],
                 labels = {"TOTALS": "Total Piano Sales"},
                 range_y=[0,400000],
                 range_color = [0,400000],
                 range_x = [1899, 2008],
                 text = 'note')
    fig.write_image("./data/img/frame_{}.png".format(i))